<a href="https://colab.research.google.com/github/nanpolend/machine-learning/blob/master/%E5%9C%96%E7%89%87%E5%8C%B9%E9%85%8DGemini123ff.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()


In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

image_matching_challenge_2025_path = kagglehub.competition_download('image-matching-challenge-2025')

print('Data source import complete.')


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip show opencv-python

Name: opencv-python
Version: 4.11.0.86
Summary: Wrapper package for OpenCV python bindings.
Home-page: https://github.com/opencv/opencv-python
Author: 
Author-email: 
License: Apache 2.0
Location: /usr/local/lib/python3.11/dist-packages
Requires: numpy
Required-by: dopamine_rl, imgaug


In [ ]:
import pandas as pd
import numpy as np # 常用於處理矩陣運算

# --- 模擬的預測結果 ---
# 在實際應用中，你需要用你的模型對 "測試集" (test set) 進行預測，
# 然後將預測出的 sample_id 和對應的 fundamental_matrix 填入這裡。

# 範例測試樣本 ID (來自比賽提供的 test.csv 或 sample_submission.csv)
# 格式通常類似 'batch_id;image_1_id;image_2_id' 或其他由比賽定義的格式
# *** 你需要從比賽數據中獲取真實的測試樣本 ID ***
test_sample_ids = [
    'batch1;img001;img002',
    'batch1;img003;img004',
    'batch2;img101;img105',
    # ... 包含所有測試樣本的 ID
]

# 範例預測的基礎矩陣 (Fundamental Matrices)
# 這應該是你的模型為上面每個 test_sample_id 預測出的 3x3 矩陣。
# 這裡使用隨機數據和固定數據作為範例。
# *** 你需要用你的模型預測結果替換這裡的範例數據 ***
predicted_fundamental_matrices = [
    np.random.rand(3, 3), # 範例矩陣 1 (隨機)
    np.array([[1e-8, 2e-7, 0.01],
              [5e-8, 1e-7, 0.02],
              [-0.01, -0.02, 1.0]]), # 範例矩陣 2 (模擬)
    np.eye(3) * 1e-5, # 範例矩陣 3 (接近單位矩陣，乘以小數值)
    # ... 包含對應所有測試樣本 ID 的預測矩陣
]

# --- 重要說明 ---
# 關於你提到的 'train_labels' 和 'train_thresholds':
# - 'train_labels': 在有監督學習中，這通常指訓練數據的真實標籤 (ground truth)。
#   對於圖像匹配，這可能是真實的基礎矩陣或對應點。這些是用來訓練模型的，不是提交的內容。
#   提交文件的內容是你模型對 *測試集* 的 *預測* 結果。
# - 'train_thresholds' 或一般的 'thresholds': 這通常指在模型預測過程中使用的閾值，
#   例如在 RANSAC 算法中判斷內點 (inliers) 的距離閾值，或者匹配特徵點時的置信度閾值。
#   這些是模型內部的超參數或中間步驟，通常不直接放入最終提交的基礎矩陣中。

# --- 檢查數據一致性 ---
if len(test_sample_ids) != len(predicted_fundamental_matrices):
    raise ValueError("測試樣本 ID 的數量和預測的基礎矩陣數量不匹配！")

# --- 格式化提交數據 ---
submission_list = []
for sample_id, f_matrix in zip(test_sample_ids, predicted_fundamental_matrices):
    # 1. 確保 f_matrix 是 NumPy 陣列 (如果不是的話)
    f_matrix_np = np.asarray(f_matrix)

    # 2. 檢查矩陣形狀是否為 3x3
    if f_matrix_np.shape != (3, 3):
        print(f"警告：樣本 {sample_id} 的基礎矩陣形狀不是 (3, 3)，而是 {f_matrix_np.shape}。請檢查你的預測。")
        # 可以選擇跳過此樣本或使用一個預設值，這裡僅作提醒
        # 為避免提交錯誤，可以考慮填充一個預設值或跳過
        # f_matrix_flat_str = " ".join(['0'] * 9) # 使用全零作為預設值示例

    # 3. 將 3x3 矩陣展平成 1x9 的列表
    f_matrix_flat = f_matrix_np.flatten().tolist()

    # 4. 將列表中的每個數字轉換為字串
    f_matrix_str_list = [str(num) for num in f_matrix_flat]

    # 5. 使用空格將 9 個數字字串連接起來
    f_matrix_formatted_string = ' '.join(f_matrix_str_list)

    # 6. 將 sample_id 和格式化後的矩陣字串加入列表
    submission_list.append({
        'sample_id': sample_id,
        'fundamental_matrix': f_matrix_formatted_string
    })

# --- 生成 Pandas DataFrame ---
submission_df = pd.DataFrame(submission_list)

# --- 確保列的順序符合比賽要求 (通常是 'sample_id', 'fundamental_matrix') ---
submission_df = submission_df[['sample_id', 'fundamental_matrix']]

# --- 保存為 CSV 文件 ---
# 比賽通常要求提交的文件名為 submission.csv
output_filename = 'submission.csv'
submission_df.to_csv(output_filename, index=False) # index=False 表示不將 DataFrame 的索引寫入 CSV 文件

print(f"提交文件 '{output_filename}' 已成功生成。")
print("文件包含 {} 行數據。".format(len(submission_df)))
print("\n文件內容預覽 (前 5 行):")
print(submission_df.head())

提交文件 'submission.csv' 已成功生成。
文件包含 3 行數據。

文件內容預覽 (前 5 行):
              sample_id                                 fundamental_matrix
0  batch1;img001;img002  0.6573009028058712 0.003155558259978375 0.2642...
1  batch1;img003;img004  1e-08 2e-07 0.01 5e-08 1e-07 0.02 -0.01 -0.02 1.0
2  batch2;img101;img105          1e-05 0.0 0.0 0.0 1e-05 0.0 0.0 0.0 1e-05
